In [1]:
import sys, os
import numpy as np
from math import *
import math
from PIL import Image
from matplotlib import pyplot
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
import cv2
import random
import dlib

In [2]:
def bb_intersection_over_union(box1, box2):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
	box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(box1Area + box2Area - interArea)
	# return the intersection over union value
	return iou

In [3]:
def calculate_iou(gtbox,HOGbox):
  no_counter = 0
  text = img_file + "\n" + str(num_faces) + "," + str(len(HOGbox)) + "\n"
  for boxA in gtbox:
    if len(HOGbox)==0:
      text += ','.join([str(elem) for elem in boxA]) + '/' + "None" + '/' + "0.0" + '/' + "0.0" + '\n'
      no_counter += 1
    else:
      test_iou = []
      for boxB in HOGbox:
        #print(boxB[0])
        test_iou.append(bb_intersection_over_union(boxA, boxB[0]))
      #FN
      if max(test_iou) == 0.0:
        text += ','.join([str(elem) for elem in boxA]) + "/" + "None" + '/' + "0.0" + "/" + str(max(test_iou)) + "\n"
        no_counter += 1
      #TP
      else:
        for i,u in enumerate(test_iou):
          if u==max(test_iou):
            index=i
            text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in HOGbox[i][0]]) + "/" + str(HOGbox[i][1]) + "/" + str(max(test_iou)) + "\n"
            break
    """#FP
    for boxC in HOGbox:
      test_iou=[]
      for boxD in gtbox:
        test_iou.append(bb_intersection_over_union(boxC, boxD))
        if max(test_iou)==0.0:
          text += 'None' + '/' + ','.join([str(elem) for elem in boxC])' + '/' + str(correct_iou) + '\n' """
  
  print(text)
  return text

In [4]:
rect_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/rect/FDDB-fold-10-rectList.txt'
iou_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/iou/FDDB-fold-10-NN-HOG-IOU.txt'

min_range= -0.9999952922024287
max_range= 3.933456913120312

with open(rect_filename) as f:
	lines = [line.rstrip('\n') for line in f]

counter = 0
f = open(iou_filename,'w')
while counter < len(lines):
  img_file = lines[counter] 
  image = cv2.imread(img_file)
  num_faces = int(lines[counter+1])
  gtbox = []
  for j in range(num_faces):
    gt = [int(s) for s in lines[counter+j+2].split(',')]
    gtbox.append(gt)

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  detector = dlib.get_frontal_face_detector()
  faces, scores, idx = detector.run(image, 1, -1)
  HOGrect = []
  Confidence = []
  print(faces)
  for i, s in enumerate(scores):
    #n = (s-min_range) / (max_range-min_range)
    Confidence.append(s)
  for face in faces:
    x1 = face.left() 
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    HOG = [x1,y1,x2,y2]
    HOGrect.append(HOG)
  HOGbox = list(zip(HOGrect,Confidence))
  #print(HOGbox)

  text = calculate_iou(gtbox,HOGbox)
  
  f.write(text)
  counter = counter + num_faces + 2

try:
  print(no_counter)
except: 
  pass
f.close()

rectangles[[(254, 113) (328, 188)], [(-20, 30) (63, 105)], [(357, 161) (400, 204)], [(179, 96) (254, 171)], [(294, 218) (356, 281)], [(74, 50) (117, 94)], [(305, 210) (341, 246)], [(73, 191) (135, 253)], [(371, 146) (415, 190)], [(161, 118) (197, 154)]]
/content/drive/MyDrive/paper_face_recognitioin/FDDB-images/2002/08/31/big/img_18008.jpg
4,10
0,1,69,109/-20,30,63,105/0.8022241746254388/0.5315846994535519
164,66,239,177/179,96,254,171/0.16662106858700776/0.48031496062992124
254,79,333,188/254,113,328,188/1.3131402172578688/0.6477272727272727
356,124,425,212/371,146,415,190/-0.9358211458102272/0.32504012841091495

rectangles[[(91, 68) (199, 175)], [(55, 199) (98, 242)], [(-15, 226) (25, 262)], [(152, 245) (204, 297)]]
/content/drive/MyDrive/paper_face_recognitioin/FDDB-images/2002/08/22/big/img_249.jpg
1,4
75,10,192,192/91,68,199,175/1.9037153115115606/0.49288590604026844

rectangles[[(86, 136) (175, 226)], [(12, -7) (55, 41)], [(56, 15) (242, 201)], [(181, 337) (233, 389)], [(3, 90) (